In [1]:
import tensorflow as tf
import os
import numpy as np
import random

random.seed(42)

tfrecords_dir='data/AWID3_tfrecords'

In [2]:
tfrecords = [os.path.join(tfrecords_dir, file) for file in os.listdir(tfrecords_dir) if file.endswith('.tfrecord')]
epochs = 2

random.shuffle(tfrecords)
test_ratio = 0.8
train_size = int(len(tfrecords) * test_ratio)
train_data = tfrecords[:train_size]
test_data = tfrecords[train_size:]


print("Training set:")
train_names = [t.split('/')[-1].split('.')[0] for t in train_data]
for t in train_names:
    print(t)

print("\nTest set:")
test_names = [t.split('/')[-1].split('.')[0] for t in test_data]
for t in test_names:
    print(t)

Training set:
AWID3_tfrecords\Deauth_29
AWID3_tfrecords\Evil_Twin_52
AWID3_tfrecords\RogueAP_38
AWID3_tfrecords\(Re)Assoc_31
AWID3_tfrecords\RogueAP_33
AWID3_tfrecords\Deauth_22
AWID3_tfrecords\Evil_Twin_63
AWID3_tfrecords\Kr00k_40
AWID3_tfrecords\Evil_Twin_73
AWID3_tfrecords\Kr00k_48
AWID3_tfrecords\(Re)Assoc_23
AWID3_tfrecords\RogueAP_36
AWID3_tfrecords\Kr00k_49
AWID3_tfrecords\Kr00k_44
AWID3_tfrecords\Evil_Twin_55
AWID3_tfrecords\Kr00k_37
AWID3_tfrecords\Kr00k_53
AWID3_tfrecords\Disas_32
AWID3_tfrecords\Deauth_24
AWID3_tfrecords\Evil_Twin_39
AWID3_tfrecords\Deauth_27
AWID3_tfrecords\Kr00k_52
AWID3_tfrecords\(Re)Assoc_29
AWID3_tfrecords\Disas_37
AWID3_tfrecords\Kr00k_51
AWID3_tfrecords\Evil_Twin_40
AWID3_tfrecords\Disas_40
AWID3_tfrecords\Disas_31
AWID3_tfrecords\Kr00k_55
AWID3_tfrecords\Evil_Twin_38
AWID3_tfrecords\Disas_33
AWID3_tfrecords\Kr00k_31
AWID3_tfrecords\Evil_Twin_61
AWID3_tfrecords\Evil_Twin_50
AWID3_tfrecords\Evil_Twin_44
AWID3_tfrecords\Kr00k_34
AWID3_tfrecords\Krack_25

In [ ]:
sequence_length = 64
sequence_shift = 56
batch_size = 50
n_feautes = 40

import data_utils

raw_train_ds = tf.data.TFRecordDataset(train_data)
train_ds = raw_train_ds.map(data_utils.parse_record).shuffle(100000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

raw_test_ds = tf.data.TFRecordDataset(test_data)
test_ds = raw_train_ds.map(data_utils.parse_record).batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_seq_ds = data_utils.create_sequential_dataset(train_data, seq_length=sequence_length, seq_shift=sequence_shift, batch_size=batch_size)
test_seq_ds = data_utils.create_sequential_dataset(test_data, seq_length=sequence_length, seq_shift=sequence_shift, batch_size=batch_size, shuffle=False)

In [4]:
import cnn_lstm

cnn_lstm_model = cnn_lstm.CNN_LSTM_model()

cnn_lstm_model.fit(
        train_seq_ds,
        epochs=epochs,
        callbacks = [cnn_lstm.checkpoint_callback],
    )

Epoch 1/2
    795/Unknown 685s 598ms/step - accuracy: 0.8226 - loss: nan - precision: 0.1438 - recall: 0.0056
Epoch 1: accuracy improved from -inf to 0.82639, saving model to saved_models/cnn_lstm.keras
795/795 ━━━━━━━━━━━━━━━━━━━━ 685s 598ms/step - accuracy: 0.8226 - loss: nan - precision: 0.1438 - recall: 0.0056


c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Epoch 2/2
795/795 ━━━━━━━━━━━━━━━━━━━━ 0s 565ms/step - accuracy: 0.8281 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2: accuracy improved from 0.82639 to 0.82706, saving model to saved_models/cnn_lstm.keras
795/795 ━━━━━━━━━━━━━━━━━━━━ 623s 565ms/step - accuracy: 0.8281 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00


In [5]:
cnn_lstm_model.summary()

cnn_lstm_model.evaluate(test_seq_ds)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, None, 128)      │        25,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, None, 128)      │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 256)      │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, None, 256)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, None, 256)      │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, None, 256)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, None, 240)      │        61,680 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, None, 240)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, None, 120)      │        28,920 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, None, 120)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, None, 1)        │           121 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,255,845 (12.42 MB)

 Trainable params: 1,085,281 (4.14 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,170,564 (8.28 MB)

164/164 ━━━━━━━━━━━━━━━━━━━━ 53s 67ms/step - accuracy: 0.8320 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00


c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


[nan, 0.8313971757888794, 0.0, 0.0]

In [6]:
import cnn1d

cnn1d_model = cnn1d.CNN1D_model()


cnn1d_model.fit(
        train_ds,
        epochs=epochs,
        callbacks = [cnn1d.checkpoint_callback],
    )


Epoch 1/2
 105484/Unknown 284s 3ms/step - accuracy: 0.8926 - f1_score: 0.1345 - loss: nan - precision: 0.0186 - recall: 0.0186
Epoch 1: recall improved from -inf to 0.00205, saving model to saved_models/cnn1d.keras
105506/105506 ━━━━━━━━━━━━━━━━━━━━ 284s 3ms/step - accuracy: 0.8926 - f1_score: 0.1345 - loss: nan - precision: 0.0186 - recall: 0.0186
Epoch 2/2
105496/105506 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9277 - f1_score: 0.1344 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2: recall did not improve from 0.00205
105506/105506 ━━━━━━━━━━━━━━━━━━━━ 282s 3ms/step - accuracy: 0.9277 - f1_score: 0.1344 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00


In [7]:
cnn1d_model.summary()

cnn1d_model.evaluate(test_ds)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (None, 1, 40)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 1, 128)         │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 1, 64)          │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 1, 32)          │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 1, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 100)            │         3,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,957 (222.49 KB)

 Trainable params: 18,985 (74.16 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 37,972 (148.33 KB)

105506/105506 ━━━━━━━━━━━━━━━━━━━━ 181s 2ms/step - accuracy: 0.9274 - f1_score: 0.1351 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00


c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


[nan, 0.9271121621131897, 0.0, 0.0, 0.13587220013141632]

In [9]:
import dnn


dnn_model = dnn.DNN_model()

dnn_model.fit(
        train_ds,
        epochs=epochs,
        callbacks = [dnn.checkpoint_callback],
    )


Epoch 1/2
 105500/Unknown 238s 2ms/step - accuracy: 0.9269 - f1_score: 0.1345 - loss: 0.0678 - precision: 0.3228 - recall: 0.0139
Epoch 1: f1_score improved from -inf to 0.13587, saving model to saved_models/dnn.keras
105506/105506 ━━━━━━━━━━━━━━━━━━━━ 238s 2ms/step - accuracy: 0.9269 - f1_score: 0.1345 - loss: 0.0678 - precision: 0.3228 - recall: 0.0139
Epoch 2/2


c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


105500/105506 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9278 - f1_score: 0.1344 - loss: 0.0684 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2: f1_score did not improve from 0.13587
105506/105506 ━━━━━━━━━━━━━━━━━━━━ 234s 2ms/step - accuracy: 0.9278 - f1_score: 0.1344 - loss: 0.0684 - precision: 0.0000e+00 - recall: 0.0000e+00


In [10]:
dnn_model.summary()

dnn_model.evaluate(test_ds)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 30)             │         1,230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30)             │           120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 20)             │           620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 20)             │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 16)             │           336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 12)             │           204 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 12)             │            48 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 6)              │            78 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 6)              │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,456 (21.32 KB)

 Trainable params: 2,643 (10.32 KB)

 Non-trainable params: 168 (672.00 B)

 Optimizer params: 2,645 (10.34 KB)

105506/105506 ━━━━━━━━━━━━━━━━━━━━ 155s 1ms/step - accuracy: 0.9274 - f1_score: 0.1350 - loss: 0.0756 - precision: 0.0000e+00 - recall: 0.0000e+00


c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


[0.07589024305343628, 0.9271121621131897, 0.0, 0.0, 0.13587220013141632]

In [ ]:
from tensorflow.keras.layers import Input, Dense, TimeDistributed, Concatenate

input_shape = (sequence_length, n_feautes)
input = Input(input_shape)

# making per packet models take  
td_cnn1d_model = TimeDistributed(cnn1d_model)
td_dnn_model = TimeDistributed(dnn_model)

base_classifiers = [
    cnn_lstm_model
    td_cnn1d_model,
    td_dnn_model,
]

# disabling training of base classifiers
for bc in base_classifiers:
    bc.trainable=False

base_classifiers_outputs = [bc(input) for bc in base_classifiers]
combined_output = Concatenate()(base_classifiers_outputs)

# Logistic Regression for outputs of every timestep
y = TimeDistributed(Dense(1, activation='sigmoid'))(combined_output)

stacked_model = tf.keras.Model(input, y)

metaclassifier_epochs = 2

loss = tf.keras.losses.BinaryCrossentropy(
    from_logits=True
)

stacked_model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)


In [13]:
stacked_model.fit(
    train_seq_ds,
    epochs=metaclassifier_epochs,
)

Epoch 1/2
795/795 ━━━━━━━━━━━━━━━━━━━━ 253s 80ms/step - accuracy: 0.8273 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/2


c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


795/795 ━━━━━━━━━━━━━━━━━━━━ 245s 78ms/step - accuracy: 0.8265 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00


In [15]:
stacked_model.summary()

stacked_model.evaluate(test_seq_ds)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 64, 40)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 64, 1)     │  1,085,281 │ input_layer_4[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_9  │ (None, 64, 1)     │     18,985 │ input_layer_4[0]… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_10 │ (None, 64, 1)     │      2,811 │ input_layer_4[0]… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 64, 3)     │          0 │ sequential[1][0], │
│ (Concatenate)       │                   │            │ time_distributed… │
│                     │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_11 │ (None, 64, 1)     │          4 │ concatenate_1[0]… │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,107,091 (4.22 MB)

 Trainable params: 4 (16.00 B)

 Non-trainable params: 1,107,077 (4.22 MB)

 Optimizer params: 10 (44.00 B)

164/164 ━━━━━━━━━━━━━━━━━━━━ 57s 83ms/step - accuracy: 0.8321 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00


c:\Users\filip\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


[nan, 0.8313971757888794, 0.0, 0.0]